# WRANGLING NAV AND PRICE DATA FROM EXCEL SHEETS

Wrangling nav and price data with python from 293 Excel Sheets for algorithmic analysis. Decided to work with Python to clean the data and then when the data is formatted, use Pandas to clean out the data faster.

In [446]:
#IMPORTING AN XLS FILE WITH PANDAS

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [618]:
#Searching through the file directory for the files and import files.

import os
from pprint import pprint

path = os.getcwd() #get the current working directory
files = [] #A new list to store files

for r, d, f in os.walk(path): # r=root, d=directories, f = files #Find the files within directories in the root folder
    for file in f:  #Iterating through all discovered files, append the file names to the list
        if ('.xls' in file) or ('.xlsx' in file):
            files.append(os.path.join(r, file))       
print(len(files))       
pprint(files[60])

293
('C:\\Users\\Kenny\\Documents\\Code\\Nav\\NAV Sheets\\NAV as at 20th February '
 '2015 (1).xlsx')


## SORTING 2016 TO 2019 DATA

I noticed a similarity in the formatting of the Excel documents between 2016 and 2019. Writing a custom function to scrape the data from the sheet and format it, so I can join the data on Pandas. Wrote a custom function to extract data

In [626]:
#A CUSTOM FUNCTIO TO PREPROCESS SHEET DATA

def extract_data(file_name):
    book = xlrd.open_workbook(file_name)  #importing single sheet
    sheet_names = list(book.sheet_names())

    if 'Sheet1' in sheet_names:
        sheet = book.sheet_by_name('Sheet1') #Get the sheet in the excel workbook
    elif 'Data' in sheet_names:
        sheet = book.sheet_by_name('Data')  #Get the sheet in the excel workbook

    data = [] #create a data list to hold the data
    add=0 #A MARKER TO NOTE IF THE FIRST ROW IS EMPTY

    for row in range(sheet.nrows):  #selecting rows from the excel sheet
        row_value = sheet.row_values(row)[0:9] #Select the values in each row

        if row == 0 and len(''.join(row_value)) == 0: #check if the first row is empty
            add = 1                                  #if empty, move all rows upwards by 1

        elif row == 1+add: #check for the row with date data, note it's lower if the first row isnt empty
            date1 = (''.join(row_value[3:4])).split('as ')[1] #get the dates
            date2 = (''.join(row_value[6:7])).split('as ')[1] #get the second set of dates

        if row == 2+add:
            row_value[3]  = row_value[3] + ' ' + date1  #add date to the NAV e.g NAV as at Week End 24 November 2017
            row_value[4]  = row_value[4] + ' ' + date1
            row_value[5]  = row_value[5] + ' ' + date1
            row_value[6]  = row_value[6] + ' ' + date2
            row_value[7]  = row_value[7] + ' ' + date2
            row_value[8]  = row_value[8] + ' ' + date2

            data.append(row_value) #Append values to sheet

        if row > 2+add:
            first,fourth = row_value[0], row_value[4] #check if the 1st three cells of a row are empty
            if first != "" and fourth !="":
                #print(row_value[1:9]) #Print values of data
                #fund_names.append(row_value[2])
                data.append(row_value[0:9]) #Append values to sheet
    return data

In [627]:
#SORTING THROUGH EXCEL SHEETS FROM 2017 - PRESENT

import xlrd

listed = []
i=0
#book = xlrd.open_workbook(files[1]) #Open the excel workbook
type1 = ['2017','2018','2019']
#type1 = ['2016']
for file in files:
    name_list = (file.replace('-',' ')).split()
    for name in name_list:
        if name in type1:
            i+=1
            #print(file)
            listed.append(file)

print(i)

19


(??)Problem, only 19 files are selected. :(

## PROCEEDING WITH THE 19 FILES LISTED

In [628]:
#for file in listed:
    #data = extract_data(file)
    #datadf = pd.DataFrame(data[1:], columns=data[0])
   # datadf = datadf.drop(["S/N","NAME OF THE FUND MANAGER"],axis=1)
  #  if datadf:

### TESTING WITH 2 FILES

In [629]:
data1 = extract_data("NAV Sheets/NAV-as-at-14th-June-2019-1.xls")
data2 = extract_data("NAV Sheets/NAV-as-at-24th-March-2016.xlsx")

# Creating Dataframe from list, appending first row as columns
data1df = pd.DataFrame(data1[1:], columns=data1[0])
data1df = data1df.drop(["S/N","NAME OF THE FUND MANAGER"],axis=1)

# Creating Dataframe from list, appending first row as columns
data2df = pd.DataFrame(data2[1:], columns=data2[0])
data2df = data2df.drop(["S/N","NAME OF THE FUND MANAGER"],axis=1)

In [630]:
data1df.tail(5)

,FUND,"NAV at Week Ended June 7, 2019","% on Total at Week Ended June 7, 2019","Unit Price at Week Ended June 7, 2019","NAV at Week Ended June 14, 2019","% on Total at Week Ended June 14, 2019","Unit Price at Week Ended June 14, 2019"
88,Vetiva S & P Nig. Sovereign Bond ETF,6.312004e+08,0.112695,179.30,6.312004e+08,0.118361,179.30
89,New Gold ETF,7.050000e+08,0.125871,4700.00,4.500000e+08,0.084383,3000.00
90,Lotus Capital Halal ETF,4.820000e+08,0.086056,10.00,4.820000e+08,0.090383,10.00
91,Stanbic IBTC ETF 30 Fund,4.677905e+08,0.083519,119.95,4.620759e+08,0.086647,119.95
92,SIAML ETF 40,7.175761e+08,0.128116,120.92,7.101524e+08,0.133166,120.92


In [632]:
data2df.sample(5)

,FUND,"NAV at Week Ended March 18, 2016","% on Total at Week Ended March 18, 2016","Unit Price at Week Ended March 18, 2016","NAV at Week Ended March 24, 2016","% on Total at Week Ended March 24, 2016","Unit Price at Week Ended March 24, 2016"
12,FBN Money Market Fund,9.495970e+10,0.531910,100.00,9.550837e+10,0.534510,100.00
30,Skye Shelter Fund,2.338269e+09,0.051328,100.00,2.339224e+09,0.051351,100.00
62,Stanbic IBTC ETF 30 Fund,8.121313e+08,0.221661,72.00,8.121313e+08,0.221630,72.00
6,Nigeria Global Investment Fund,1.146049e+08,0.009061,1.91,1.148468e+08,0.009032,1.96
60,New Gold ETF,3.600000e+08,0.098257,2400.00,3.508500e+08,0.095747,2339.00


In [633]:
#MERGED THE TWO TEST DATASETS
twodf = pd.merge(data2df, data1df, on='FUND', how='outer')
twodf.sample(10)

,FUND,"NAV at Week Ended March 18, 2016","% on Total at Week Ended March 18, 2016","Unit Price at Week Ended March 18, 2016","NAV at Week Ended March 24, 2016","% on Total at Week Ended March 24, 2016","Unit Price at Week Ended March 24, 2016","NAV at Week Ended June 7, 2019","% on Total at Week Ended June 7, 2019","Unit Price at Week Ended June 7, 2019","NAV at Week Ended June 14, 2019","% on Total at Week Ended June 14, 2019","Unit Price at Week Ended June 14, 2019"
94,Kedari Investment Fund (KIF),NaN,NaN,NaN,NaN,NaN,NaN,2.897602e+08,0.000513,131.200000,2.904288e+08,0.000515,131.500000
26,BGL Sapphire Fund,0.000000e+00,0.000000,0.0000,0.000000e+00,0.000000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN
16,Meristem Money Market Fund,2.072076e+08,0.001161,10.0000,2.455018e+08,0.001374,10.0000,8.293411e+08,0.001469,10.000000,8.350624e+08,0.001481,10.000000
72,Chapel Hill Denham Money Market Fund,NaN,NaN,NaN,NaN,NaN,NaN,1.624068e+09,0.002877,100.000000,1.639849e+09,0.002909,100.000000
38,FBN Heritage Fund,4.392337e+09,0.188432,107.4200,4.399609e+09,0.188108,107.6200,2.824846e+09,0.116833,145.110000,2.810868e+09,0.116776,144.410000
47,PACAM Balanced Fund,1.831055e+08,0.007855,1.0724,1.848728e+08,0.007904,1.0712,1.228651e+08,0.005082,1.292207,1.229536e+08,0.005108,1.293439
85,Stanbic IBTC Guaranteed Investment Fund,NaN,NaN,NaN,NaN,NaN,NaN,9.835405e+09,0.125452,256.300000,9.928342e+09,0.124775,256.880000
68,AXA Mansard Money Market Fund,NaN,NaN,NaN,NaN,NaN,NaN,2.630043e+10,0.046595,1.000000,2.629589e+10,0.046643,1.000000
31,Union Homes REITS,1.212660e+10,0.266197,45.2200,1.212421e+10,0.266153,45.2200,9.856409e+09,0.216363,40.700000,9.855688e+09,0.216337,40.700000
60,New Gold ETF,3.600000e+08,0.098257,2400.0000,3.508500e+08,0.095747,2339.0000,7.050000e+08,0.125871,4700.000000,4.500000e+08,0.084383,3000.000000


In [634]:
export_excel = twodf.to_excel (r'C:\Users\Kenny\Desktop\NAVs\export_dataframe.xlsx', index = None, header=True) 
#Exporting to an excel file

## TRYING ANOTHER APPROACH

In [635]:
stanbic = [] #eXTRACTING DATA FROM ONLY ONE FUND

for file_name in files[0:150]:
    print(file_name)
    book = xlrd.open_workbook(file_name)  #importing single sheet
    sheet_names = list(book.sheet_names())

    if 'Sheet1' in sheet_names:
        sheet = book.sheet_by_name('Sheet1') #Get the sheet in the excel workbook
    elif 'Data' in sheet_names:
        sheet = book.sheet_by_name('Data')  #Get the sheet in the excel workbook

    for row in range(sheet.nrows):  #selecting rows from the excel sheet
        row_value = sheet.row_values(row)[0:9] #Select the values in each row
        #print(row_value)
        i=0
        for cell in row_value:
            if ("Stanbic IBTC Nigerian Equity Fund".strip()).split() == (str(cell)).strip().split():
                #print(row_value)
                stanbic.append(row_value[i:]) #Append values to sheet
            i=i+1             
stanbic

C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\Copy-of-NAV-as-at-4th-March-2016.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV 2 Dec 2011.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price 191211.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price 30 Dec 2011.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price 30 Dec 2011.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price as at 10th February 2012 WITH CHARTS.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer price as at 9 March 2012.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price as at April 13, 2012 WITH CHARTS..xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price as at April 20, 2012 WITH CHARTS.(1).xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price as at April 27, 2012 WITH CHARTS..xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV and Offer Price as at April 6, 2012 WITH CHARTS..xls
C:\Use

C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at 8th May 2015.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at 9th January 2015.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at 9th MAY  2014.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at 9th November 2012.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at August 15, 2014.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at August 1st, 2014 (1).xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at August 1st, 2014.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at August 29 2014.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV as at July 4 2014.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV-as-at-10th-August-2018.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV-as-at-10th-February-2017.xls
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV-as-at-10th-June-2016.xlsx
C:\Users\Kenny\Documents\Code\Nav\NAV Sheets\NAV-as-at-10th-March-2017.xls
C:\Users\Kenny\

[['Stanbic  IBTC Nigerian Equity Fund',
  6884686318.55,
  6550.8,
  6991655279.06,
  6726.07],
 ['Stanbic  IBTC Nigerian Equity Fund',
  'Stanbic IBTC Asset Mgt. Limited',
  12771319830.55,
  7139.38,
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  12674106819.02,
  7091.65,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13176861579.95,
  7380.62,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13176861579.95,
  7380.62,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13092955836.6,
  7385.09,
  12571306504.29,
  7313.43,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13092955836.6,
  7385.09,
  12571306504.29,
  7313.43,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  11917340384.12,
  7256.45,
  11768912709.36,
  7198.04,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  11768912709.36,
  7198.04,
  12304068739.97,
  7532.06,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equit

In [636]:
stanbic

[['Stanbic  IBTC Nigerian Equity Fund',
  6884686318.55,
  6550.8,
  6991655279.06,
  6726.07],
 ['Stanbic  IBTC Nigerian Equity Fund',
  'Stanbic IBTC Asset Mgt. Limited',
  12771319830.55,
  7139.38,
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  12674106819.02,
  7091.65,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13176861579.95,
  7380.62,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13176861579.95,
  7380.62,
  '',
  '',
  '',
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13092955836.6,
  7385.09,
  12571306504.29,
  7313.43,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  13092955836.6,
  7385.09,
  12571306504.29,
  7313.43,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  11917340384.12,
  7256.45,
  11768912709.36,
  7198.04,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equity Fund',
  11768912709.36,
  7198.04,
  12304068739.97,
  7532.06,
  '',
  ''],
 ['Stanbic  IBTC Nigerian Equit